# Getting exchange information from CoinMarketCap

## Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import ast
import os
import time
import csv
import json
import requests

In [3]:
df = pd.read_csv('../../data/csv/crypto_information.csv')
print(df.shape)
df.head()

(1000, 19)


,id,name,symbol,slug,num_market_pairs,date_added,tags,circulating_supply,total_supply,price,volume_24h,percent_change_1h,percent_change_24h,percent_change_7d,market_cap,category,exchange_id,exchange_name,exchange_slug
0,6535,NEAR Protocol,NEAR,near-protocol,443,2020-08-11T00:00:00.000Z,"platform,ai-big-data,staking,coinbase-ventures...",1.123916e+09,1.209633e+09,4.001789,1.896790e+08,-0.295862,0.695971,6.834548,4.497676e+09,AI,"[270, 7680, 21, 521, 407, 1188, 9867, 294, 333...","['Binance', 'DOEX', 'BTCC', 'Bybit', 'DigiFine...","['binance', 'doex', 'btcc', 'bybit', 'digifine..."
1,8916,Internet Computer,ICP,internet-computer,291,2021-03-23T00:00:00.000Z,"platform,ai-big-data,smart-contracts,polychain...",4.698829e+08,5.223637e+08,7.984759,6.524906e+07,-0.279245,4.461639,10.682161,3.751901e+09,AI,"[270, 21, 521, 10361, 407, 1188, 9867, 294, 33...","['Binance', 'BTCC', 'Bybit', 'CoinUp.io', 'Dig...","['binance', 'btcc', 'bybit', 'coinup-io', 'dig..."
2,3773,Artificial Superintelligence Alliance,FET,artificial-superintelligence-alliance,370,2019-03-02T00:00:00.000Z,"cosmos-ecosystem,ai-big-data,collectibles-nfts...",2.520000e+09,2.630547e+09,1.298860,2.768840e+08,1.142639,9.698876,14.320927,3.273126e+09,AI,"[270, 7680, 21, 9200, 521, 10361, 407, 1188, 9...","['Binance', 'DOEX', 'BTCC', 'Zedcex Exchange',...","['binance', 'doex', 'btcc', 'zedcex-exchange',..."
3,22974,Bittensor,TAO,bittensor,86,2023-03-06T05:20:00.000Z,"ai-big-data,distributed-computing,oracles,dao,...",7.365602e+06,7.365602e+06,277.748222,6.626699e+07,-0.296370,4.985333,5.307924,2.045783e+09,AI,"[270, 521, 9867, 333, 6137, 513, 943, 1653, 57...","['Binance', 'Bybit', 'KCEX', 'LBank', 'Toobit'...","['binance', 'bybit', 'kcex', 'lbank', 'toobit'..."
4,5690,Render,RENDER,render,465,2020-06-11T00:00:00.000Z,"art,media,vr-ar,ai-big-data,distributed-comput...",3.924605e+08,5.322208e+08,5.157754,8.485562e+07,-0.694359,3.914209,7.146705,2.024215e+09,AI,"[270, 521, 9867, 294, 333, 6137, 513, 943, 929...","['Binance', 'Bybit', 'KCEX', 'OKX', 'LBank', '...","['binance', 'bybit', 'kcex', 'okx', 'lbank', '..."


#### Getting the exchanges' ID's

In [4]:
# Convert each string in the 'exchange_id' column to a list
df.loc[:, 'exchange_id'] = df['exchange_id'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

In [6]:
# Concat all lists of exchanges
exchange_ids_list = np.concatenate(df['exchange_id'].values.reshape(-1, 1))

# Flatten the list of lists
flattened_exchange_ids = [item for sublist in exchange_ids_list for item in sublist]

# Keeping the unique values
unique_exchanges_ids = set(flattened_exchange_ids)

# Convert it to list
unique_exchanges_ids = list(unique_exchanges_ids)

print('Number of exchanges:', len(unique_exchanges_ids))

Number of exchanges: 523


# Get api request

Test with one Coin

In [12]:
# Define tu clave de API
api_key = os.getenv('COINMARKETCAP_MARCO_KEY')

# Define el ID de la exchange (Binance)
exchange_id = '270'

# Define la URL base de la API
url = 'https://pro-api.coinmarketcap.com/v1/exchange/info'

# Parámetros de la solicitud
params = {
    'id': exchange_id,
    'aux': 'urls,logo,description,date_launched,notice'  # Puedes añadir más campos si lo necesitas
}

# Cabeceras de la solicitud (incluyendo tu clave de API)
headers = {
    'Accepts': 'application/json',
    'X-CMC_PRO_API_KEY': api_key
}

# Realiza la solicitud GET
response = requests.get(url, headers=headers, params=params)

# Convierte la respuesta en JSON
data = response.json()

# Muestra los resultados
print(data)


{'status': {'timestamp': '2024-09-14T19:25:36.679Z', 'error_code': 0, 'error_message': None, 'elapsed': 8, 'credit_count': 1, 'notice': None}, 'data': {'270': {'id': 270, 'name': 'Binance', 'slug': 'binance', 'description': "## What Is Binance?\n\nBinance is the world’s largest crypto exchange by trading volume, with $76 billion daily trading volume on Binance exchange as of August 2022, and 90 million customers worldwide. The platform has established itself as a trusted member of the crypto space, where users can buy, sell and store their digital assets, as well as access over 350 cryptocurrencies listed and thousands of trading pairs. The Binance ecosystem now comprises of Binance Exchange, Labs, Launchpad, Info, Academy, Research, Trust Wallet, Charity, NFT and more.\n\n\n## Who Are the Binance Founders?\n\nThe global company was co-founded in China by Changpeng Zhao and Yi He. A Chinese-Canadian developer and business executive, Changpeng Zhao, who goes by CZ, is the company's CEO.

In [13]:
# Lista de IDs únicos de exchanges
# Define la URL base de la API
url = 'https://pro-api.coinmarketcap.com/v1/exchange/info'

# Cabeceras de la solicitud (incluyendo tu clave de API)
headers = {
    'Accepts': 'application/json',
    'X-CMC_PRO_API_KEY': api_key
}

# Inicializamos una lista para guardar los datos de cada exchange
all_data = []

# Iteramos sobre los IDs de exchanges
for exchange_id in unique_exchanges_ids:
    # Parámetros de la solicitud para cada ID
    params = {
        'id': exchange_id,
        'aux': 'urls,logo,description,date_launched,notice'
    }
    
    # Realiza la solicitud GET
    response = requests.get(url, headers=headers, params=params)
    
    # Si la respuesta es exitosa (código 200)
    if response.status_code == 200:
        data = response.json()
        all_data.append(data)
        
        print(f"Datos de la exchange con ID {exchange_id} obtenidos con éxito.")
    else:
        print(f"Error al obtener datos de la exchange con ID {exchange_id}. Código de estado: {response.status_code}")
    
    # Esperar 7 segundos antes de la próxima solicitud
    time.sleep(7)

# Guardamos los datos en un archivo JSON
with open('../../data/json/exchanges_data.json', 'w') as json_file:
    json.dump(all_data, json_file, indent=4)

Datos de la exchange con ID 4098 obtenidos con éxito.
Datos de la exchange con ID 16 obtenidos con éxito.
Datos de la exchange con ID 21 obtenidos con éxito.
Datos de la exchange con ID 24 obtenidos con éxito.
Datos de la exchange con ID 34 obtenidos con éxito.
Datos de la exchange con ID 36 obtenidos con éxito.
Datos de la exchange con ID 37 obtenidos con éxito.
Datos de la exchange con ID 42 obtenidos con éxito.
Datos de la exchange con ID 50 obtenidos con éxito.
Datos de la exchange con ID 4152 obtenidos con éxito.
Datos de la exchange con ID 61 obtenidos con éxito.
Datos de la exchange con ID 4159 obtenidos con éxito.
Datos de la exchange con ID 68 obtenidos con éxito.
Datos de la exchange con ID 70 obtenidos con éxito.
Datos de la exchange con ID 73 obtenidos con éxito.
Datos de la exchange con ID 80 obtenidos con éxito.
Datos de la exchange con ID 89 obtenidos con éxito.
Datos de la exchange con ID 10330 obtenidos con éxito.
Datos de la exchange con ID 8282 obtenidos con éxito.
D

UnicodeEncodeError: 'charmap' codec can't encode character '\u010d' in position 2208: character maps to <undefined>

In [14]:
# Guardamos los datos en un archivo CSV
csv_columns = ['id', 'name', 'slug', 'logo', 'description', 'date_launched', 'notice', 'weekly_visits', 'spot_volume_usd']

csv_file = '../../data/csv/exchanges_data.csv'

try:
    with open(csv_file, 'w', newline='', encoding='utf-8') as csvfile:  # Añadimos el encoding utf-8
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        
        # Recorremos los datos obtenidos para escribirlos en el CSV
        for exchange in all_data:
            for key, exchange_data in exchange['data'].items():
                writer.writerow({
                    'id': exchange_data.get('id'),
                    'name': exchange_data.get('name'),
                    'slug': exchange_data.get('slug'),
                    'logo': exchange_data.get('logo'),
                    'description': exchange_data.get('description'),
                    'date_launched': exchange_data.get('date_launched'),
                    'notice': exchange_data.get('notice'),
                    'weekly_visits': exchange_data.get('weekly_visits'),
                    'spot_volume_usd': exchange_data.get('spot_volume_usd')
                })
except IOError:
    print("Error al escribir en el archivo CSV")

print("Datos guardados exitosamente en JSON y CSV.")


Datos guardados exitosamente en JSON y CSV.
